# Building Agents with Tools in Python - Exercise

This notebook will guide you through building AI agents that can call external tools using OpenAI's function calling capabilities.

## Learning Objectives
- Understand OpenAI function calling mechanics
- Learn to create tool schemas with proper validation
- Practice forcing specific tool execution with `tool_choice`
- Implement error handling for tool calls
- Work with financial calculation tools

## Tools to Implement
1. **Compound Interest Calculator**: Calculate investment growth over time
2. **Currency Converter**: Convert between USD, EUR, and GBP (already implemented as reference)

## Instructions
Look for `# TODO: YOUR CODE HERE` comments and follow the hints to complete the implementation.

In [ ]:
# Environment Setup and Imports
import os, sys, json, argparse
from datetime import datetime
from dotenv import load_dotenv
from openai import OpenAI

# Load environment variables and initialize OpenAI client for Vocareum
load_dotenv()
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=os.getenv("OPENAI_API_KEY")
)

# System prompt for the financial assistant
SYSTEM = "You are a helpful financial assistant. Always produce concise, factual answers."

print("✅ Environment setup complete!")

In [ ]:
# Tool Implementations

def calculate_compound_interest(principal: float, rate: float, time: int, compounds_per_year: int = 12) -> dict:
    """
    Calculate compound interest for financial planning.
    Return dict: {"principal": float, "final_amount": float, "total_interest": float, "rate": float, "time": int}
    """
    # TODO: YOUR CODE HERE
    # Hint: Use the compound interest formula: A = P(1 + r/n)^(nt)
    # Where: P=principal, r=rate/100, n=compounds_per_year, t=time
    # Calculate final_amount using the formula above
    # Then calculate total_interest = final_amount - principal
    # Return a dictionary with all the required fields (see docstring)
    pass

def convert_currency(amount: float, from_code: str, to_code: str) -> dict:
    """
    Convert amount using a small static table for USD/EUR/GBP.
    Return dict: {"amount": float, "rate": float, "from": str, "to": str}
    """
    RATES = {
        "USD": {"USD": 1.0, "EUR": 0.92, "GBP": 0.78},
        "EUR": {"USD": 1.09, "EUR": 1.0, "GBP": 0.85},
        "GBP": {"USD": 1.28, "EUR": 1.18, "GBP": 1.0},
    }
    rate = RATES.get(from_code, {}).get(to_code)
    if rate is None:
        raise ValueError(f"Unsupported pair {from_code}->{to_code}")
    return {"amount": round(amount * rate, 2), "rate": rate, "from": from_code, "to": to_code}

print("✅ Tool functions defined!")

In [ ]:
# OpenAI Tool Schemas

TOOLS = [
    {
        "type": "function",
        "function": {
            "name": "calculate_compound_interest",
            "description": "Calculate compound interest for financial planning and investment analysis.",
            "parameters": {
                # TODO: YOUR CODE HERE
                # Hint: Define a JSON schema with:
                # - type: "object"
                # - properties: dict with principal, rate, time, compounds_per_year
                # - required: list of required parameter names
                # - additionalProperties: False
                # Each property should have "type" and "description"
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "convert_currency",
            # TODO: YOUR CODE HERE
        },
    },
]

# Map tool names to Python callables
FUNCTIONS = {
    # TODO: YOUR CODE HERE
}

print("✅ Tool schemas defined!")

In [ ]:
# Core Tool Execution Logic

def agentic_tool_call(tool_name: str, user_content: str, tool_args: dict):
    """Direct tool call via tool_choice with agentic response generation."""
    # 1) Ask the model to call ONLY the specified tool (no ambiguity)
    first = client.chat.completions.create(
        # TODO: YOUR CODE HERE
        # Hint: Use tool_choice parameter to direct the specific tool
        # Format: {"type": "function", "function": {"name": tool_name}}
        temperature=0.0,
    )
    msg = first.choices[0].message
    tool_call = msg.tool_calls[0] if msg.tool_calls else None

    # 2) Execute the Python function
    if tool_call is None:
        raise RuntimeError("Model did not issue the expected tool call.")
    name = tool_call.function.name
    # Merge any model-suggested args with CLI-provided args (CLI wins)
    model_args = json.loads(tool_call.function.arguments or "{}")
    call_args = {**model_args, **tool_args}
    result = FUNCTIONS[name](**call_args)

    # 3) Send tool result back and get final assistant reply
    messages = [
        {"role": "system", "content": SYSTEM},
        {"role": "user", "content": user_content},
        {"role": "assistant", "content": msg.content or "", "tool_calls": [tool_call.model_dump()]},
        # TODO: YOUR CODE HERE
        # Hint: Add a tool message with the result
        # Format: {"role": "tool", "tool_call_id": tool_call.id, "name": name, "content": json.dumps(result)}
    ]
    final = client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=0.2)
    return final.choices[0].message.content

print("✅ Core execution logic defined!")

In [ ]:
# Test Functions for Interactive Use

def test_compound_interest(principal, rate, time, compounds_per_year=12):
    """Test the compound interest calculator with given parameters."""
    prompt = f"Calculate compound interest for ${principal} at {rate}% annual rate for {time} years. Explain the result clearly."
    response = # TODO: YOUR CODE HERE
    print(response)

def test_currency_converter(amount, from_code, to_code):
    """Test the currency converter with given parameters."""
    prompt = f"Convert {amount} {from_code} to {to_code} and explain the rate used in one sentence."
    response = # TODO: YOUR CODE HERE
    print(response)

print("✅ Test functions ready!")

In [ ]:
# Example Usage - Test Your Implementation

# First test the currency converter (this should work as reference)
print("🧪 Testing Currency Converter (Reference Implementation)...")
test_currency_converter(125, "USD", "EUR")

print("\n" + "="*60 + "\n")

# Then test your compound interest implementation
print("🧪 Testing Compound Interest Calculator (Your Implementation)...")
# TODO: Uncomment the line below once you've implemented the function
# test_compound_interest(1000, 5, 10, 12)